In [1]:
import pandas as pd
import glob
import numpy as np

In [13]:
# not all years are completed yet from part 2a
# # Recursive query - extra /**/ added to string
# q = "Data/**/final_tmdb_data_201*.csv.gz" 
# chunked_files = sorted(glob.glob(q, recursive=True)) 
# # Showing the first 5 
# chunked_files[:10]

chunked_files = ['Data\\final_tmdb_data_2011.csv.gz',
 'Data\\final_tmdb_data_2012.csv.gz']

In [11]:
## Loading all files as df and appending to a list
df_list = []
for file in chunked_files:
    temp_df = pd.read_csv(file, index_col=0)
    df_list.append(temp_df)
    
## Concatenating the list of dfs into 1 combined
df = pd.concat(df_list)
df = df.drop(df.index[0]).reset_index(drop=True)
df = df.drop(df.index[0]).reset_index(drop=True)
df.head(3)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0.0,NaN,NaN,0.0,"[{'id': 99, 'name': 'Documentary'}]",https://www.leightonpierce.com/cms/index.php?o...,239465.0,en,50 Feet of String,"""The slow and subtle repeated rhythms of daily...",...,0.0,52.0,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,NaN,50 Feet of String,0.0,6.000,3.0,NaN
1,0.0,/oJiBH1aSFv0gOKwb34I7CCKqgRE.jpg,NaN,7750000.0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 27, '...",http://www.thewickertreemovie.com/,79544.0,en,The Wicker Tree,Director Robin Hardy's reimagining of his eeri...,...,0.0,96.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Accept our sacrifice,The Wicker Tree,0.0,3.875,44.0,R
2,0.0,NaN,NaN,5000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.spythemovie.com/,323867.0,en,Spy,SPYING TURNS TO SECURITY DETAIL AS A YOUNG COU...,...,0.0,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Spy,0.0,7.000,1.0,NaN


In [12]:
## Columns to exclude - Keeping Status and companies
drop_cols = ['backdrop_path','backdrop_path','original_title','overview',
                 'poster_path','tagline','id','homepage', #'status','production_companies'
                 'production_countries','video','spoken_languages',
            'original_language']
df = df.drop(columns=drop_cols)

## Use .notna() to get True if it belongs to a collection
df['belongs_to_collection'] = df['belongs_to_collection'].notna()

## Function to get just the genre names as a list 
import json
def get_genre_name(x):
    x = x.replace("'",'"')
    x = json.loads(x)
    
    genres = []
    for genre in x:
        genres.append(genre['name'])
    return genres
## Use ourn function and exploding the new column
df['genres_list'] = df['genres'].apply(get_genre_name)
df_explode = df.explode('genres_list')

## save unique genres
unique_genres = df_explode['genres_list'].dropna().unique()

## Manually One-Hot-Encode Genres
for genre in unique_genres:
    df[f"Genre_{genre}"] = df['genres'].str.contains(genre,regex=False)    


## Drop original genre cols
df = df.drop(columns=['genres','genres_list'])


#### Fixing Certification
## Checking Certification values
df['certification'].value_counts(dropna=False)
# fix extra space certs
df['certification'] = df['certification'].str.strip()

## fix certification col
repl_cert = {'UR':'NR',
             'Not Rated':'NR',
             'Unrated':'NR',
             '-':'NR',
             '10':np.nan,
             'ScreamFest Horror Film Festival':'NR'}
df['certification'] = df['certification'].replace(repl_cert)
df['certification'].value_counts(dropna=False)


#### Converting year to sep features
## split release date into 3 columns
new_cols = ['year','month','day']
df[new_cols] = df['release_date'].str.split('-',expand=True)
df[new_cols] = df[new_cols].astype(float)

## drop original feature
df = df.drop(columns=['release_date'])

## Save only Released status
df = df.loc[ df['status'] == 'Released']
df = df.drop(columns=['status'])

## filter out financials that don't have financial data
df = df.loc[(df['budget'] >0 ) & (df['revenue']>0)]

df=df.drop(columns = 'production_companies')
df

,adult,belongs_to_collection,budget,popularity,revenue,runtime,title,vote_average,vote_count,certification,...,Genre_Romance,Genre_Fantasy,Genre_Adventure,Genre_Music,Genre_History,Genre_War,Genre_TV Movie,year,month,day
3,0.0,False,45000000.0,11.605,24000000.0,120.0,The Rum Diary,5.868,1075.0,R,...,False,False,False,False,False,False,False,2011.0,10.0,13.0
4,0.0,True,36000000.0,23.541,189712432.0,84.0,Gnomeo & Juliet,5.860,1766.0,G,...,False,False,False,False,False,False,False,2011.0,1.0,13.0
5,0.0,False,163000000.0,24.973,174822325.0,119.0,Cowboys & Aliens,5.584,4608.0,PG-13,...,False,False,False,False,False,False,False,2011.0,7.0,29.0
7,0.0,False,110000000.0,62.423,299268508.0,127.0,Real Steel,6.967,7307.0,PG-13,...,False,False,False,False,False,False,False,2011.0,9.0,28.0
10,0.0,True,130000000.0,51.213,554987477.0,90.0,Puss in Boots,6.524,3356.0,PG,...,False,True,True,False,False,False,False,2011.0,10.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6033,0.0,False,190000.0,1.863,62000.0,117.0,Chapters,6.600,5.0,NaN,...,False,False,False,False,False,False,False,2012.0,12.0,7.0
6050,0.0,True,240000.0,2.602,2800000.0,127.0,Pizza,6.875,52.0,NaN,...,True,False,False,False,False,False,False,2012.0,10.0,19.0
6061,0.0,False,16.0,5.767,16786016.0,120.0,Love 911,7.200,54.0,NaN,...,True,False,False,False,False,False,False,2012.0,12.0,19.0
6087,0.0,False,2723311.0,2.591,1424291.0,96.0,The Giant King,6.000,7.0,PG,...,False,True,True,False,False,False,False,2012.0,10.0,4.0
